In [2]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [16]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
kaufman_sets = [
'c85de504-96cf-4856-b7f3-398e6bf1e37b',
'c50006d4-a98d-4366-bd5b-650fee1696ee',
'43586c61-a527-43d3-b95a-1bc063fa1fb7',
'e2b336fb-dda9-4eff-9125-0603cc99916e',
'f41f9b4b-9f13-4e25-abed-13fcf99b5a73',
'1ed849f5-cf49-4190-84a9-161f13906c21',
'c33e4a8d-cd5e-4bf2-8973-88a46efd2173',
'd5fd5f93-8ffc-4bdc-8db9-1457d119248b',
]
my_exp = kaufman_sets[1]

print my_exp
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




c50006d4-a98d-4366-bd5b-650fee1696ee
workflow 2324ad76-ff37-4157-8bcc-3ce72b7dace9 released to project
workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
42
42


In [14]:
## This part is only for making non file objects "in submission"
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        for raw_data in store[a_type]:
            print '    SKIPPING FILES', a_type, raw_data['status'], raw_data['uuid']
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print '    DELETED DATA', a_type ,raw_data['uuid']
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            #ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab c85de504-96cf-4856-b7f3-398e6bf1e37b
quality_metric_fastqc in review by lab 4fef9536-0d29-42fe-923b-92c89eacfef6
quality_metric_fastqc in review by lab d5cf0b6b-1555-49aa-81a4-cddc6a16d40b
biosample_cell_culture in review by lab d6708984-239a-4993-b31d-ec11cecc7827
protocol in review by lab 9e13a395-8ea8-43f1-ac3c-7bea650f4672
protocol in review by lab 42a9b094-5092-41c9-a1c7-fa25aa901050
experiment_seq in review by lab 7a883bde-c9f5-4410-9453-8f7ffacbc4c3
    SKIPPING FILES file_fastq uploaded 775bce7f-29df-4bae-9169-dbb921c50aef
    SKIPPING FILES file_fastq released 3aae0107-7e7b-469c-af91-411f22effd7a
biosample in review by lab 1fbb281e-ee66-4c0b-9a44-d933f5be2824
workflow_run_awsem in review by lab deb90371-2a72-4aa8-a2b1-d21f3097bae0
workflow_run_awsem in review by lab 5f71d02d-f5c9-4ecb-baf3-b7493058f93c
workflow_run_awsem in review by lab 56346bfb-3915-48b2-a171-6487efe5264e
workflow_run_awsem in review by lab 8053d223-b8bf-4545-a007-38

In [15]:
counter = 0
for a_type in store:
    if a_type.startswith('file') and not a_type.startswith('file_set'):
        continue

    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "submission in progress"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)

print counter

experiment_set_replicate in review by lab c85de504-96cf-4856-b7f3-398e6bf1e37b
quality_metric_fastqc in review by lab 4fef9536-0d29-42fe-923b-92c89eacfef6
quality_metric_fastqc in review by lab d5cf0b6b-1555-49aa-81a4-cddc6a16d40b
biosample_cell_culture in review by lab d6708984-239a-4993-b31d-ec11cecc7827
protocol in review by lab 9e13a395-8ea8-43f1-ac3c-7bea650f4672
protocol in review by lab 42a9b094-5092-41c9-a1c7-fa25aa901050
experiment_seq in review by lab 7a883bde-c9f5-4410-9453-8f7ffacbc4c3
biosample in review by lab 1fbb281e-ee66-4c0b-9a44-d933f5be2824
workflow_run_awsem in review by lab deb90371-2a72-4aa8-a2b1-d21f3097bae0
workflow_run_awsem in review by lab 5f71d02d-f5c9-4ecb-baf3-b7493058f93c
workflow_run_awsem in review by lab 56346bfb-3915-48b2-a171-6487efe5264e
workflow_run_awsem in review by lab 8053d223-b8bf-4545-a007-38543635428d
12
